## Turtle graph analysis

Notebook obsahuje kód na grafovú vizualizáciu sekvencií riešení úloh Želví Grafiky [https://www.umimeprogramovat.cz/zelvi-grafika] a analýzu dát na základe grafovej charakteristiky jednotlivých hier. 

### Data parsing and preprocessing

Časť načítania, dekódovania a prieskumnej analýzy logov - smelo preskočiť rovno nižšie ku grafom

In [1]:
import pandas as pd

In [2]:
item_df = pd.read_csv("data/new/turtle_item.txt", delimiter=";")
item_df["id"] = item_df["id"].apply(int)
item_df.set_index("id", inplace=True)

In [3]:
item_df.head()

,name,instructions,startingBlocks,solution,medianTime,visible
id,,,,,,
1,Půlka čtverce,"[""text"",""Co dodat, snaz se.""]","<xml id=""startBlocks"" style=""display: none""> ...",for (var count = 0 count < 1 count++) { Tur...,0,1
2,Pětiúhelník,"[""text"",""Pětiúhelník je podobný jako čtverec, ...",NaN,for (var count = 0 count < 5 count++) { Tur...,0,1
3,Oprav čtverec,"[""text"",""Stačí změnit počet opakování a délku ...","<xml xmlns=""http://www.w3.org/1999/xhtml""><var...",Turtle.moveForward(100)Turtle.turnRight(90)Tur...,0,1
4,Dodělej čtverec,"[""text"",""Přidej další příkazy pro pohyb dopřed...","<xml xmlns=""http://www.w3.org/1999/xhtml""><var...",Turtle.moveForward(100)Turtle.turnRight(90)Tur...,0,1
5,Dodělej trojúhelník,"[""text"",""Přidej další příkazy pro zatočení dop...","<xml xmlns=""http://www.w3.org/1999/xhtml""><var...",Turtle.moveForward(100)Turtle.turnRight(60)Tur...,0,1


In [4]:
log_df = pd.read_csv("data/new/turtle_log.txt", delimiter=";")
log_df.describe()

,id,user,item,correct,moves,responseTime
count,149609.000000,1.496090e+05,149609.000000,149609.000000,149609.0,1.496090e+05
mean,74805.000000,1.775931e+06,22.042016,0.096799,-1.0,1.468145e+05
std,43188.542549,3.012258e+05,17.834914,0.295685,0.0,4.471455e+05
min,1.000000,2.000000e+00,1.000000,0.000000,-1.0,0.000000e+00
25%,37403.000000,1.691300e+06,6.000000,0.000000,-1.0,3.285000e+04
50%,74805.000000,1.849861e+06,13.000000,0.000000,-1.0,7.260000e+04
75%,112207.000000,1.965020e+06,35.000000,0.000000,-1.0,1.550000e+05
max,149609.000000,2.054128e+06,63.000000,1.000000,-1.0,5.688755e+07


In [92]:
log_df.head()

,id,user,item,answer,correct,moves,responseTime,time
64694,64695,1880458,25,"Turtle.turnLeft(70, 'Sbz}[2C*(Y2WNLB[h7R9');\n",0,-1,90600,2018-02-22 07:58:51
64695,64696,1880458,25,"Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\n",0,-1,100000,2018-02-22 07:59:00
64696,64697,1880458,25,"Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,111050,2018-02-22 07:59:11
64697,64698,1880458,25,"Turtle.turnLeft(25, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,125100,2018-02-22 07:59:25
64698,64699,1880458,25,"Turtle.turnLeft(30, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,136100,2018-02-22 07:59:36


In [7]:
from dateutil import parser
log_df = log_df[log_df["time"].apply(parser.parse) >= parser.parse("2018-02-22")]

In [8]:
import base64

In [9]:
def decode_base64(data):
    """Decode base64, padding being optional.

    :param data: Base64 data as an ASCII byte string
    :returns: The decoded byte string.

    """
    try:
        missing_padding = len(data) % 4
        if missing_padding != 0:
            data += b'='* (4 - missing_padding)

        return base64.decodestring(data)
    except Exception:
        return None

In [ ]:
log_df["answer"] = log_df["answer"].apply(decode_base64)

In [11]:
log_df["answer"].values

array(["Turtle.turnLeft(70, 'Sbz}[2C*(Y2WNLB[h7R9');\n",
       "Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\n",
       "Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\nTurtle.moveForward(100, 'L^$@NV7Mtk7ZkYjz.U3p');\n",
       ...,
       "Turtle.moveForward(50, '/6/AEHd7c9/aVz@Gy)E)');\nTurtle.turnRight(45, '!:88}1XXE@*OJlK{mKat');\nTurtle.turnLeft(45, '*1Oy|1`srePFae)iyjW/');\n",
       "Turtle.moveForward(50, '/6/AEHd7c9/aVz@Gy)E)');\nTurtle.turnRight(45, '!:88}1XXE@*OJlK{mKat');\nTurtle.turnLeft(45, '*1Oy|1`srePFae)iyjW/');\n",
       "Turtle.penColour('#6633ff', '5#ZY{sDB-M_TzdmMNykS');\nTurtle.moveForward(100, '3=9.{^wFd1`3x;lUZD#]');\nTurtle.turnRight(90, '#zU?R{{a|M@j1n#gV;kf');\nTurtle.penColour('#ff0000', ';4`SlrE]wskk7$_0Jlih');\n"],
      dtype=object)

In [12]:
log_df.loc[64711, "answer"]

'var x, y;\n\n/**\n * Popite tuto funkci...\n */\nfunction diamant(x, y) {\nif(--window.LoopTrap == 0) throw "Infinite loop.";\n  for (var count = 0; count < y; count++) {\n  if(--window.LoopTrap == 0) throw "Infinite loop.";\n    mnoho_C3_BAheln_C3_ADk(x);\n    Turtle.turnRight(360 / y, \'W#}iGxw8z@MEIy79[]bn\');\n  }\n}\n\n/**\n * Popite tuto funkci...\n */\nfunction mnoho_C3_BAheln_C3_ADk(x) {\nif(--window.LoopTrap == 0) throw "Infinite loop.";\n  for (var count2 = 0; count2 < x; count2++) {\n  if(--window.LoopTrap == 0) throw "Infinite loop.";\n    Turtle.moveForward(50, \'ny_4c+Pc=(pOzkQpq^7Y\');\n    Turtle.turnRight(360 / x, \'Zuw3+9x[NSZZXlwf2n0M\');\n  }\n}\n\n\ndiamant(5, 10);\n'

### Kanonizácia riešení

Filtrácia spusteného kódu užívateľa z javascript logov a kanonizácia riešení do tvaru <prikaz(parameter)> zachovávajúc odsadenie pre reprezentáciu zanorenia. 

In [14]:
def filter_commands(command):
    return command.strip().startswith("Turtle") or command.strip().startswith("for")

In [15]:
def modify_command(command):
    if command.strip().startswith("Turtle"):
        out = command.replace("Turtle.move", "").replace("Turtle.turn", "")
        out = out[:out.find(",")]+")"
    elif command.strip().startswith("for"):
        # for happened
        out = command[command.find("< ")+2:]
        out = command[:command.find("for")] + "Repeat(%s)" % out[:out.find(";")]
    else:
        out = command
    return out

In [16]:
import os


def cannonize_answers(answers_series):
    commands_df = answers_series.apply(lambda s: str(s).split("\n")) \
                                .apply(lambda commands: tuple(filter(filter_commands, commands))) \
                                .apply(pd.Series) \
                                .fillna("") \
                                .applymap(modify_command)
    commands_s = commands_df.apply(lambda commands: os.linesep.join(commands[commands.apply(len) > 0]), axis=1)
    return commands_s

In [17]:
cannonize_answers(log_df["answer"])

64694                                              Left(70)
64695                                              Left(20)
64696                                Left(20)\nForward(100)
64697                                Left(25)\nForward(100)
64698                                Left(30)\nForward(100)
64699      Left(30)\nForward(100)\nRight(120)\nForward(100)
64700     Left(30)\nForward(100)\nRight(120)\nForward(10...
64701     Left(120)\nForward(100)\nRight(120)\nForward(1...
64702     Repeat(4)\n  Left(120)\n  Forward(100)\n  Righ...
64703     Repeat(4)\n  Left(120)\n  Forward(100)\n  Righ...
64704                                          Forward(100)
64705                Repeat(4)\n  Forward(100)\n  Right(90)
64706     Repeat(4)\n  Forward(100)\n  Right(90)\n  Forw...
64707     Repeat(4)\n  Forward(100)\n  Right(90)\nForwar...
64708     Repeat(4)\n  Forward(100)\n  Right(90)\nForwar...
64709     Repeat(4)\n  Repeat(4)\n    Forward(100)\n    ...
64710     Repeat(8)\n  Repeat(4)\n    Fo

In [54]:
def get_seq_id(df):
    return df.index.values

### Graph drawing utilities

Inicializácia množiny vrcholov a generácia grafu

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import igraph as jgraph
from jgraph import *
jgraph.__version__

py.tools.set_credentials_file(username='stmichal', api_key='MpxBW1OVR6aOfQoaynfc')

In [30]:
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
import matplotlib.pyplot as plt

def generate_nx_graph(edge_set):
    
    G = nx.DiGraph()

    all_v = {v[0] for v in edge_set}.union({v[1] for v in edge_set})
    
    map(G.add_node, all_v)
    G.add_edges_from(list(edge_set))
    
    return G

In [31]:
def make_annotations(pos, text, font_size=8, font_color='rgb(0,0,0)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = go.Annotations()
    M = max([p[1] for p in pos.values()])
    for k in pos.keys():
        annotations.append(
            go.Annotation(
                text=k, # or replace labels with a different list for the text within the circle  
                x=pos[k][0], y=pos[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=True)
        )
    return annotations  

In [36]:
import numpy as np

### Vizualizácia grafu

In [37]:
def draw_tree(g_dict, game_id, correct_answs):
    edges_list = g_dict.keys()
    G = generate_nx_graph(edges_list)
    pos = graphviz_layout(G, prog='dot')
    
    final_pos = dict()
    for k in correct_answs.values:
        try:
            final_pos[k] = pos[k]
            
        except KeyError:
            pass
#             print "Final State for Game %s not found in responses" % (game_id)
            
    pos_reduced = {k: pos[k] for k in set(pos.keys()) - set(final_pos.keys())}
    
    Xn = [p[0] for p in pos_reduced.values()]
    Yn = [p[1] for p in pos_reduced.values()]

    Xe = []
    Ye = []    
    widths = []
    
    for edge in G.edges:
        Xe += (pos[edge[0]][0], pos[edge[1]][0])
        Ye += (pos[edge[0]][1], pos[edge[1]][1])
        widths += [g_dict[(edge[0], edge[1])]]
        
    Xe = pd.Series(Xe)
    Ye = pd.Series(Ye)

    widths = pd.Series(sum(map(lambda x: [x, x], widths), []))

    # Create Plotly Traces
    lines = [go.Scatter(x=Xe[widths == w],
                       y=Ye[widths == w],
                       mode='lines+text',
                       opacity=0.6,
                       line=dict(color='rgb(%s,%s,%s)' % tuple([255/1.2-int(float(w)/max(widths)*255/2)]*3), width=2))
             for w in widths.unique()]
    dots = go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='fail',
                      marker=dict(symbol='dot',
                                    size=18, 
                                    color='#6175c1',    #'#DB4551', 
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                      text=pos_reduced.keys(),
                      hoverinfo='text',
                      opacity=0.6
                      )
    final_dots = go.Scatter(x=[p[0] for p in final_pos.values()],
                            y=[p[1] for p in final_pos.values()],
                            mode='markers',
                            name='success',
                            marker=dict(symbol='dot',
                                    size=18, 
                                    color='#006600',    #'#DB4551', 
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                            text=final_pos.keys(),
                            hoverinfo='text',
                            opacity=0.8
                      )

    # Create Text Inside the Circle via Annotations
    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )

    layout = dict(title= 'Transition graph for Game #%s' % game_id,  
                  font=dict(size=12),
                  showlegend=False,
                  xaxis=go.XAxis(axis),
                  yaxis=go.YAxis(axis),
                  width=1000,
                  height=1000,
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
                  plot_bgcolor='rgb(256,256,256)'          
                  )
    # Plot a tree
    data=go.Data(lines+[dots, final_dots])
        
    fig=dict(data=data, layout=layout)
    return fig

In [39]:
def filter_edges_below_lim(edge_dict, lim=2):
    return {k: edge_dict[k] for k in edge_dict.keys() if edge_dict[k] >= lim}

In [125]:
def get_edges_for_game(game_id, game_logs):
    edge_dict = dict()
    seq_ids = game_logs.groupby(by="user", sort=False).apply(get_seq_id)
    for seq in seq_ids:
        seq_contents = [" "] + list(game_logs.loc[seq, "answer"].values)
        seq_contents = filter(lambda s: not pd.isnull(s) and len(s) > 0, seq_contents)
        for seq_i in range(1, len(seq_contents)):
            try:
                edge_dict[(seq_contents[seq_i-1], seq_contents[seq_i])] += 1
            except KeyError:
                edge_dict[(seq_contents[seq_i-1], seq_contents[seq_i])] = 1
                
    return edge_dict

In [ ]:
# to get pygraphviz running, we need to compile and install graphviz backend:
# sudo apt-get install graphviz libgraphviz-dev graphviz-dev pkg-config
# pip install pygraphviz

In [162]:
# id of the game is visible in the suffix of the game url, e.g:
# https://www.umimeprogramovat.cz/zelvi-grafika-jednoduche/12 lays under game_id = 12
game_id = 55
# min # of transitions through the edge to be visible
edges_support_min = 3

# collect each game sequences from one player, and construct a graph upon it
game_logs = log_df[log_df["item"] == game_id]
game_logs["answer"] = cannonize_answers(log_df[log_df["item"] == game_id]["answer"])
game_logs.sort_values(by="time", inplace=True)

# construct a graph
edge_dict = get_edges_for_game(game_id, game_logs)

correct_answs = game_logs[game_logs["correct"] == 1]["answer"]

edge_dict_filtered = filter_edges_below_lim(edge_dict, lim=edges_support_min)

fig = draw_tree(edge_dict_filtered, game_id, correct_answs)

py.iplot(fig, file_id='Game %s edge support %s' % (game_id, edges_support_min))

/home/michal/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/michal/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [47]:
pd.Series(edge_dict.values()).describe()

count    260.000000
mean       2.076923
std        3.262316
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       31.000000
dtype: float64

## Analytická časť

### Prísnejšia kanonizácia

Z grafov vidno, že často sa frekventované cesty mieriace do cieľa líšia poradím v ktorom užívateľ ladil parametre programu. Niektorí užívatelia najskôr riešia "kalibráciu" počtu opakovaní cyklov a la +-1, niektorí riešia najskôr dĺžku kroku korytnačky, niektorí kalibrujú úhol natočenia. 

Takéto modifikácie by sa mali zvažovať všetky ako ekvivalentne správne. Pritom pár krokov naviac za účelom kalibrácie je jednoznačne cesta k správnemu cieľu. Napr. z hry 55 vyššie vidno, že takéto kalibrácie majú za následok väčšinu vetvenia v grafe.

Z tejto motivácie som implementoval aj prísnejšiu kanonizáciu zanedbávajúcu všetky parametre metód, tj zohľadňujúcu iba poradie príkazov v sekvencii.

In [73]:
def modify_command_no_params(command):
    if command.strip().startswith("Turtle"):
        out = command.replace("Turtle.move", "").replace("Turtle.turn", "")
        out = out[:out.find("(")]
    elif command.strip().startswith("for"):
        out = command[command.find("< ")+2:]
        out = command[:command.find("for")] + "Repeat"
    else:
        out = command
    return out

In [74]:
def cannonize_answers_no_params(answers_series):
    commands_df = answers_series.apply(lambda s: str(s).split("\n")) \
                                .apply(lambda commands: tuple(filter(filter_commands, commands))) \
                                .apply(pd.Series) \
                                .fillna("") \
                                .applymap(modify_command_no_params)
    commands_s = commands_df.apply(lambda commands: os.linesep.join(commands[commands.apply(len) > 0]), axis=1)
    return commands_s

In [163]:
# id of the game is visible in the suffix of the game url, e.g:
# https://www.umimeprogramovat.cz/zelvi-grafika-jednoduche/12 lays under game_id = 12
game_id = 55
# min # of transitions through the edge to be visible
edges_support_min = 3

# collect each game sequences from one player, and construct a graph upon it
game_logs = log_df[log_df["item"] == game_id]
game_logs["answer"] = cannonize_answers_no_params(log_df[log_df["item"] == game_id]["answer"])
game_logs.sort_values(by="time", inplace=True)
seq_ids = game_logs.groupby(by="user", sort=False).apply(get_seq_id)

# construct a graph
edge_dict = dict()  
for seq in seq_ids:
    seq_contents = [" "] + list(game_logs.loc[seq, "answer"].values)
    seq_contents = filter(lambda s: not pd.isnull(s) and len(s) > 0, seq_contents)
    for seq_i in range(1, len(seq_contents)):
        try:
            edge_dict[(seq_contents[seq_i-1], seq_contents[seq_i])] += 1
        except KeyError:
            edge_dict[(seq_contents[seq_i-1], seq_contents[seq_i])] = 1

correct_answs = game_logs[game_logs["correct"] == 1]["answer"]

edge_dict_filtered = filter_edges_below_lim(edge_dict, lim=edges_support_min)

fig = draw_tree(edge_dict_filtered, game_id, correct_answs)

py.iplot(fig, file_id='Game %s edge support %s' % ("Zajímavý vzor", edges_support_min))    

/home/michal/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/michal/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [114]:
pd.Series(edge_dict.values()).describe()

count    123.000000
mean       4.390244
std       14.769616
min        1.000000
25%        1.000000
50%        1.000000
75%        1.500000
max      141.000000
dtype: float64

### Kanonizácia: záver

V grafe pre rovnakú hru vyššie vidíme podstatnú redukciu množstva častých stavov prísnejšom kanonizáciou, so stratou informácie o parametroch metód. Nenapadá ma ale, ako táto informácia (o parametroch) môže byť užitočná pre generáciu nápovied, čo mal byť hlavný účel klasifikácie.

## Výber vhodnej metriky pre reprezentáciu skúsenosti užívateľa

Nižšie skúmame koreláciu niekoľkých latentných vlastností s celkovým časom riešenia úlohy, za účelom ohodnotenia schopností užívateľa nejakou spojitou metrikou. Predpokladáme pritom, že užitočné metriky budú s časom riešenia korelovať významne.

In [113]:
log_df.head(10)

,id,user,item,answer,correct,moves,responseTime,time
64694,64695,1880458,25,"Turtle.turnLeft(70, 'Sbz}[2C*(Y2WNLB[h7R9');\n",0,-1,90600,2018-02-22 07:58:51
64695,64696,1880458,25,"Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\n",0,-1,100000,2018-02-22 07:59:00
64696,64697,1880458,25,"Turtle.turnLeft(20, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,111050,2018-02-22 07:59:11
64697,64698,1880458,25,"Turtle.turnLeft(25, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,125100,2018-02-22 07:59:25
64698,64699,1880458,25,"Turtle.turnLeft(30, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,136100,2018-02-22 07:59:36
64699,64700,1880458,25,"Turtle.turnLeft(30, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,186500,2018-02-22 08:00:27
64700,64701,1880458,25,"Turtle.turnLeft(30, 'Sbz}[2C*(Y2WNLB[h7R9');\n...",0,-1,232850,2018-02-22 08:01:13
64701,64702,1880458,25,"Turtle.turnLeft(120, 'Sbz}[2C*(Y2WNLB[h7R9');\...",0,-1,313850,2018-02-22 08:02:34
64702,64703,1880458,25,for (var count = 0; count < 4; count++) {\nif(...,0,-1,337400,2018-02-22 08:02:58
64703,64704,1880458,25,for (var count = 0; count < 4; count++) {\nif(...,1,-1,492900,2018-02-22 08:05:40


### Korelácia dĺžky cesty s časom riešenia

In [110]:
no_answers = log_df.groupby(by=["user", "item"]).count()["answer"]
time_lapses = log_df.groupby(by=["user", "item"])["responseTime"].sum()
answers_times_df = pd.DataFrame(no_answers).join(pd.DataFrame(time_lapses))
answers_times_df.head(10)

answer  responseTime
user item                      
2    4          1         14300
12   3          1         35600
     4          2        182800
     8          2        175150
     11         2         64950
     24         6        444750
     33         1         12950
     35         3        740600
     36         4        114750
     41         4        183950

In [111]:
answers_times_df.corr()

,answer,responseTime
answer,1.000000,0.622723
responseTime,0.622723,1.000000


### Korelácia priemernej vzialenosti riešenia od najkratšej cesty k riešeniu s časom riešenia

Komplikovanejšia metrika, ktorá predpokladá, že najlepší kandidáti nasledujú najkratšiu cestu riešenia - vzdialenosť uzlu od tejto cesty je potom parametrom každého vrcholu. Metrika priemernej vzdialenosti potom udáva ako presne užívateľ nasledoval ideálnu cestu - čím nižšia hodnota, tým presnejšie.

In [ ]:
means = []
times = []

# selection of min path to s successful solution
for game_id in log_df["item"].unique():
    print "Progress: on game %s" % game_id
    # collect each game sequences from one player, and construct a graph upon it
    game_logs = log_df[log_df["item"] == game_id]
    game_logs["answer"] = cannonize_answers(log_df[log_df["item"] == game_id]["answer"])
    game_logs.sort_values(by="time", inplace=True)
    
    correct_answs = game_logs[game_logs["correct"] == 1]["answer"]

    # construct a graph
    edge_dict = get_edges_for_game(game_id, game_logs)
    edges_list = edge_dict.keys()
    G = generate_nx_graph(edges_list)
    # reconstruct paths from root to each successful solution
    paths = [nx.shortest_path(G, source=" ", target=final_state) for final_state in correct_answs]
    try:
        # select the minimal path to solution as a pivoting path
        min_path = min(paths, key=len)
        game_opt_paths[game_id] = min_path
    except ValueError:
        game_opt_paths[game_id] = [" "]
        
    for user in game_logs["user"].unique():
        user_game_df = game_logs[game_logs["user"] == user]
        path_lengths = []
        
        # for each state of the user's solution, measure its distance towards the pivoting optimal path
        for u_state in user_game_df["answer"]:
            min_dist = 1000
            # choose the minimal one among all
            for p_state in min_path:
                try:
                    new_dist = nx.shortest_path_length(G, source=u_state, target=p_state)
                    if min_dist > new_dist:
                        min_dist = new_dist
                except:
                    continue
            if min_dist == 1000:
                min_dist = None
            path_lengths.append(min_dist)
                    
        # together aggregate users' means of the distances and their solving times
        user_mean = np.mean([p for p in path_lengths if p is not None])
        means.append(user_mean)
        
        solving_time = user_game_df["responseTime"].sum()
        times.append(solving_time)
        
# drop times for missing paths from user path to optimal path
times = [times[i] for i in range(len(list(means))) if not pd.isnull(means[i])]
means = [means[i] for i in range(len(list(means))) if not pd.isnull(means[i])]

In [201]:
pd.Series(means).describe()

count    8852.000000
mean        1.794835
std         1.750673
min         0.000000
25%         0.800000
50%         1.250000
75%         2.227597
max        28.392857
dtype: float64

In [202]:
pd.Series(times).describe()

count    8.852000e+03
mean     1.114285e+06
std      5.665794e+06
min      3.000000e+02
25%      7.988750e+04
50%      2.490750e+05
75%      7.714000e+05
max      4.216956e+08
dtype: float64

In [200]:
np.corrcoef(means, times)

array([[1.        , 0.23703293],
       [0.23703293, 1.        ]])

### Záver: korelácie

Korelácia dĺžky cesty s časom riešenia potvrdila domnienku, že dĺžka cesty koreluje s časom riešenia naozaj významne, hoci je stále ďaleko od úplnej korelácie. Môžeme ale tvrdiť, že dĺžka cesty nesie podstatnú informáciu o skúsenosti používateľa.

Druhá metrika ma vzhľadom na svoju komplexnosť trochu sklamala, korelácia je oveľa nižšia, hoci nie celkom bezvýznamná (23.7%). Stále je však zaujímavé pozorovanie v sumarizácii, že viac ako 50% užívateľov sa od optimálneho riešenia odklonilo v priemere len o 1.25 hrany. 

V analýze sme však uvažovali grafy s hranami s akoukoľvek podporou (aj 1), ktoré budú síce rozsiahle, ale dosť husté, resp. silne prepojené.

Mohlo by mať cenu sa ešte pohrať s parametrami, no stále relatívne nízka korelácia naznačuje, že táto cesta žiadne zázraky asi nenúka.

## Návrh klasifikácie podľa grafových dát

Preskúmal som možnosť binárnej klasifikácie (úspech <> neúspech) užívateľa podľa grafovej reprezentácie priechodov danej hry. Navrhovaný prístup:

* Každý uzol grafu je ohodnotený Success-rate-om udávajúcim podiel úspešných riešení, ktoré prechádzajú daným uzlom, relatívne ku všetkým priechodom

* Pravdepodobnosť úspechu užívateľa je potom v každom uzle rovná Success-rate-u uzla, resp. váženým priemerom Success-rate-ov na ceste užívateľa, s určitým (silným) decay factorom.

In [90]:
pd.Series(edge_dict.values()).describe()

count    123.000000
mean       4.390244
std       14.769616
min        1.000000
25%        1.000000
50%        1.000000
75%        1.500000
max      141.000000
dtype: float64

### Záver: klasifikácia podľa grafu

Klasifikáciu podľa hrán však bohužiaľ musíme zavrhnúť pre nedostatočnú hustotu priechodov v grafe. Hoci striktnejšia kanonizácia uvedená vyššie zdvihla priemerný počet priechodov z 2.07 na 4.39, 75% hrán má stále menej, ako jeden priechod. Tento podiel sa nezvyšuje ani pri frekventovanejších hrách, tj. graf  rastie skôr "do šírky", ako "do hĺbky".

Tento problém by mohla vyriešiť ešte inteligentnejšia kanonizácia, ale asi len za cenu ďalšej straty informácie.